# ノートブック17: Titanic Top 30%達成 - 実践Kaggle完全ガイド 🏆

**学習目標**: Kaggle TitanicコンペティションでTop 30%（Public LB 0.79+）を達成

---

## 📋 このノートブックで学ぶこと

### 1. 高度な特徴量エンジニアリング ⭐⭐⭐
- Ticket prefix extraction（チケット番号から階級を推定）
- Cabin deck analysis（客室デッキ情報の活用）
- Title extraction（敬称の抽出と分類）
- Age imputation（機械学習による年齢補完）
- Feature interactions（特徴量の交互作用）

### 2. Optunaによる自動最適化 ⭐⭐⭐
- Bayesian Optimizationの実践
- LightGBM、XGBoost、CatBoostの最適化
- CVスコアを最大化するパラメータ探索

### 3. 堅牢な検証戦略 ⭐⭐
- 10-Fold Stratified Cross-Validation
- Out-of-fold predictions
- CV Score vs LB Scoreの関係理解

### 4. アンサンブル戦略 ⭐⭐⭐
- LightGBM + XGBoost + CatBoostの重み付き平均
- Blending vs Stacking
- Calibration（確率調整）

### 5. Kaggle提出戦略 ⭐
- Multiple submissions
- Leaderboard probing
- Score variance analysis

---

## 🎯 目標スコア

- **Local CV**: 0.85+
- **Public LB**: 0.79+ (Top 30%)

---

## 📚 前提知識

このノートブックを始める前に、以下を完了していることを推奨：
- ノートブック13-16（GBDT基礎）
- 基本的なPandas、NumPy、Matplotlib
- LightGBM、XGBoost、CatBoostの基本操作

---

## ⚠️ 重要な注意事項

1. **データの配置**: Kaggleから`train.csv`と`test.csv`をダウンロードし、`../data/titanic/`に配置してください
2. **計算時間**: Optunaの最適化は時間がかかります（約30-60分）
3. **再現性**: `random_state`を固定していますが、環境により若干の差が出る可能性があります

---

## 🔧 1. セットアップ

必要なライブラリをインポートし、環境を準備します。

In [ ]:
# 基本ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

# 機械学習ライブラリ
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report

# GBDT
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

# ハイパーパラメータ最適化
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances

# 設定
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# 再現性のためのシード固定
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ ライブラリのインポート完了")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"LightGBM version: {lgb.__version__}")
print(f"XGBoost version: {xgb.__version__}")
print(f"Optuna version: {optuna.__version__}")

## 📊 2. データ読み込み

Kaggle Titanicデータセットを読み込みます。

In [ ]:
# データディレクトリの作成（存在しない場合）
data_dir = Path('../data/titanic')
data_dir.mkdir(parents=True, exist_ok=True)

# データ読み込み
try:
    train = pd.read_csv(data_dir / 'train.csv')
    test = pd.read_csv(data_dir / 'test.csv')
    print("✅ データ読み込み成功")
except FileNotFoundError:
    print("⚠️ データが見つかりません。以下の手順でデータを準備してください：")
    print("1. https://www.kaggle.com/c/titanic/data にアクセス")
    print("2. train.csv と test.csv をダウンロード")
    print(f"3. {data_dir} に配置")
    # デモ用のダミーデータを作成（実際にはKaggleからダウンロード必要）
    print("\n⚠️ デモ用のダミーデータを作成します（実際のKaggle提出には使用できません）")
    from sklearn.datasets import make_classification
    X, y = make_classification(n_samples=891, n_features=10, n_informative=7, 
                                n_redundant=2, random_state=42)
    train = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(10)])
    train['Survived'] = y
    train['PassengerId'] = range(1, 892)
    test = pd.DataFrame(X[:418], columns=[f'feature_{i}' for i in range(10)])
    test['PassengerId'] = range(892, 1310)

print(f"\n📊 データセット情報:")
print(f"- Train shape: {train.shape}")
print(f"- Test shape: {test.shape}")
print(f"\n最初の5行:")
display(train.head())

# ターゲット分布
if 'Survived' in train.columns:
    print(f"\n🎯 生存率: {train['Survived'].mean():.2%}")
    print(train['Survived'].value_counts())

## 🔨 3. 高度な特徴量エンジニアリング

Top 30%を達成するための高度な特徴量を作成します。

In [ ]:
def advanced_feature_engineering(df, is_train=True):
    """
    高度な特徴量エンジニアリング
    
    Parameters:
    -----------
    df : pd.DataFrame
        入力データ
    is_train : bool
        訓練データかどうか
    
    Returns:
    --------
    pd.DataFrame : 特徴量を追加したデータフレーム
    """
    df = df.copy()
    
    # 1. Title（敬称）の抽出 ⭐⭐⭐
    if 'Name' in df.columns:
        df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
        
        # レアなタイトルをグループ化
        title_mapping = {
            'Mr': 'Mr',
            'Miss': 'Miss',
            'Mrs': 'Mrs',
            'Master': 'Master',
            'Rev': 'Rare',
            'Dr': 'Rare',
            'Col': 'Rare',
            'Major': 'Rare',
            'Mlle': 'Miss',
            'Countess': 'Rare',
            'Ms': 'Miss',
            'Lady': 'Rare',
            'Jonkheer': 'Rare',
            'Don': 'Rare',
            'Dona': 'Rare',
            'Mme': 'Mrs',
            'Capt': 'Rare',
            'Sir': 'Rare'
        }
        df['Title'] = df['Title'].map(title_mapping)
        df['Title'] = df['Title'].fillna('Rare')
    
    # 2. Family Size（家族サイズ） ⭐⭐
    if 'SibSp' in df.columns and 'Parch' in df.columns:
        df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
        df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
        
        # 家族サイズのカテゴリ化
        df['FamilySizeGroup'] = pd.cut(df['FamilySize'], 
                                        bins=[0, 1, 4, 20], 
                                        labels=['Alone', 'Small', 'Large'])
    
    # 3. Cabin Deck（デッキ情報） ⭐⭐⭐
    if 'Cabin' in df.columns:
        df['Deck'] = df['Cabin'].str[0]
        df['Deck'] = df['Deck'].fillna('Unknown')
        df['HasCabin'] = df['Cabin'].notna().astype(int)
        
        # 客室番号（存在する場合）
        df['CabinNumber'] = df['Cabin'].str.extract('([0-9]+)', expand=False)
        df['CabinNumber'] = pd.to_numeric(df['CabinNumber'], errors='coerce')
    
    # 4. Ticket prefix（チケット接頭辞） ⭐⭐
    if 'Ticket' in df.columns:
        df['TicketPrefix'] = df['Ticket'].str.split(' ').str[0]
        df['TicketPrefix'] = df['TicketPrefix'].str.replace('[^A-Za-z]', '', regex=True)
        df['TicketPrefix'] = df['TicketPrefix'].replace('', 'None')
        
        # チケット番号の長さ
        df['TicketLength'] = df['Ticket'].apply(lambda x: len(str(x)))
    
    # 5. Fare per person（1人あたりの運賃） ⭐⭐
    if 'Fare' in df.columns and 'FamilySize' in df.columns:
        df['FarePerPerson'] = df['Fare'] / df['FamilySize']
        
        # Fare のビニング
        df['FareBin'] = pd.qcut(df['Fare'].fillna(df['Fare'].median()), 
                                 q=5, labels=False, duplicates='drop')
    
    # 6. Age binning（年齢のビニング） ⭐⭐
    if 'Age' in df.columns:
        df['AgeBin'] = pd.cut(df['Age'].fillna(df['Age'].median()), 
                               bins=[0, 12, 18, 35, 60, 100], 
                               labels=['Child', 'Teen', 'Adult', 'MiddleAge', 'Senior'])
    
    # 7. Name length（名前の長さ） ⭐
    if 'Name' in df.columns:
        df['NameLength'] = df['Name'].apply(len)
    
    # 8. 交互作用特徴量 ⭐⭐⭐
    if 'Sex' in df.columns and 'Pclass' in df.columns:
        df['Sex_Pclass'] = df['Sex'].astype(str) + '_' + df['Pclass'].astype(str)
    
    if 'Age' in df.columns and 'Pclass' in df.columns:
        df['Age_Pclass'] = df['Age'].fillna(df['Age'].median()) * df['Pclass']
    
    # 9. Embarked（乗船港）の処理 ⭐
    if 'Embarked' in df.columns:
        df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
    
    print(f"✅ 特徴量エンジニアリング完了")
    print(f"   作成された特徴量数: {df.shape[1]}")
    
    return df

# 特徴量エンジニアリングの適用
train_fe = advanced_feature_engineering(train, is_train=True)
test_fe = advanced_feature_engineering(test, is_train=False)

print(f"\n📊 特徴量エンジニアリング後のshape:")
print(f"Train: {train_fe.shape}")
print(f"Test: {test_fe.shape}")
print(f"\n新しい特徴量:")
new_features = set(train_fe.columns) - set(train.columns)
for feat in sorted(new_features):
    print(f"  - {feat}")

### 3.1 Age Imputation（機械学習による年齢補完） ⭐⭐⭐

欠損している年齢を他の特徴量から予測します。

In [ ]:
def impute_age_ml(train_df, test_df):
    """
    Random Forestで年齢を補完
    """
    # 結合してまとめて処理
    combined = pd.concat([train_df, test_df], ignore_index=True, sort=False)
    
    # Age予測に使う特徴量
    age_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title']
    
    # カテゴリカル変数のエンコーディング
    for col in ['Sex', 'Embarked', 'Title']:
        if col in combined.columns:
            le = LabelEncoder()
            combined[col + '_encoded'] = le.fit_transform(combined[col].astype(str))
            age_features[age_features.index(col)] = col + '_encoded'
    
    # 欠損していないデータで学習
    known_age = combined[combined['Age'].notna()].copy()
    unknown_age = combined[combined['Age'].isna()].copy()
    
    if len(unknown_age) > 0:
        X_train = known_age[age_features].fillna(0)
        y_train = known_age['Age']
        X_test = unknown_age[age_features].fillna(0)
        
        # Random Forestで予測
        rf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
        rf.fit(X_train, y_train)
        
        predicted_ages = rf.predict(X_test)
        combined.loc[combined['Age'].isna(), 'Age'] = predicted_ages
        
        print(f"✅ {len(unknown_age)}件の年齢を補完しました")
    
    # 分割して返す
    train_len = len(train_df)
    return combined.iloc[:train_len].copy(), combined.iloc[train_len:].copy()

# 年齢補完の実行
if 'Age' in train_fe.columns:
    print(f"補完前の欠損数: {train_fe['Age'].isna().sum()}")
    train_fe, test_fe = impute_age_ml(train_fe, test_fe)
    print(f"補完後の欠損数: {train_fe['Age'].isna().sum()}")

## 🎯 4. データ前処理

モデル学習のためにデータを準備します。

In [ ]:
def prepare_data(train_df, test_df):
    """
    モデル学習用にデータを準備
    """
    # 使用しない列
    drop_cols = ['PassengerId', 'Name', 'Ticket', 'Cabin']
    if 'Survived' in train_df.columns:
        drop_cols.append('Survived')
    
    # 特徴量とターゲットの分離
    X = train_df.drop(columns=[col for col in drop_cols if col in train_df.columns])
    y = train_df['Survived'] if 'Survived' in train_df.columns else None
    X_test = test_df.drop(columns=[col for col in drop_cols if col in test_df.columns])
    
    # カテゴリカル変数の処理
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns
    
    for col in categorical_cols:
        # Label Encoding
        le = LabelEncoder()
        
        # train と test を結合してfitすることで、一貫性を保つ
        combined = pd.concat([X[col], X_test[col]], ignore_index=True)
        le.fit(combined.astype(str))
        
        X[col] = le.transform(X[col].astype(str))
        X_test[col] = le.transform(X_test[col].astype(str))
    
    # 残った欠損値を中央値で補完
    X = X.fillna(X.median())
    X_test = X_test.fillna(X_test.median())
    
    print(f"✅ データ準備完了")
    print(f"   特徴量数: {X.shape[1]}")
    print(f"   訓練データ: {X.shape[0]} samples")
    print(f"   テストデータ: {X_test.shape[0]} samples")
    
    return X, y, X_test

# データ準備
X, y, X_test = prepare_data(train_fe, test_fe)

print(f"\n📊 特徴量一覧:")
for i, col in enumerate(X.columns, 1):
    print(f"{i:2d}. {col}")

## 🚀 5. Optunaによるハイパーパラメータ最適化

LightGBM、XGBoost、CatBoostの最適なパラメータを探索します。

### 5.1 LightGBM最適化

In [ ]:
def objective_lgb(trial):
    """
    LightGBMのOptuna objective関数
    """
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'random_state': RANDOM_STATE,
        
        # Optunaで最適化するパラメータ
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
    }
    
    # Stratified K-Fold CV
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    cv_scores = []
    
    for train_idx, val_idx in cv.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model = lgb.LGBMClassifier(**params)
        model.fit(X_train, y_train, 
                 eval_set=[(X_val, y_val)],
                 callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)])
        
        preds = model.predict(X_val)
        score = accuracy_score(y_val, preds)
        cv_scores.append(score)
    
    return np.mean(cv_scores)

# LightGBM最適化の実行
print("🔍 LightGBMのハイパーパラメータ最適化を開始...")
study_lgb = optuna.create_study(direction='maximize', 
                                 sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study_lgb.optimize(objective_lgb, n_trials=50, show_progress_bar=True)

print(f"\n✅ LightGBM最適化完了")
print(f"   Best CV Score: {study_lgb.best_value:.4f}")
print(f"   Best Parameters:")
for key, value in study_lgb.best_params.items():
    print(f"     {key}: {value}")

### 5.2 XGBoost最適化

In [ ]:
def objective_xgb(trial):
    """
    XGBoostのOptuna objective関数
    """
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'use_label_encoder': False,
        'random_state': RANDOM_STATE,
        
        # Optunaで最適化するパラメータ
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
    }
    
    # Stratified K-Fold CV
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    cv_scores = []
    
    for train_idx, val_idx in cv.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model = xgb.XGBClassifier(**params)
        model.fit(X_train, y_train, 
                 eval_set=[(X_val, y_val)],
                 early_stopping_rounds=50,
                 verbose=False)
        
        preds = model.predict(X_val)
        score = accuracy_score(y_val, preds)
        cv_scores.append(score)
    
    return np.mean(cv_scores)

# XGBoost最適化の実行
print("🔍 XGBoostのハイパーパラメータ最適化を開始...")
study_xgb = optuna.create_study(direction='maximize',
                                 sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study_xgb.optimize(objective_xgb, n_trials=50, show_progress_bar=True)

print(f"\n✅ XGBoost最適化完了")
print(f"   Best CV Score: {study_xgb.best_value:.4f}")
print(f"   Best Parameters:")
for key, value in study_xgb.best_params.items():
    print(f"     {key}: {value}")

### 5.3 CatBoost最適化

In [ ]:
def objective_cat(trial):
    """
    CatBoostのOptuna objective関数
    """
    params = {
        'loss_function': 'Logloss',
        'eval_metric': 'Accuracy',
        'random_state': RANDOM_STATE,
        'verbose': False,
        
        # Optunaで最適化するパラメータ
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
    }
    
    # Stratified K-Fold CV
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    cv_scores = []
    
    for train_idx, val_idx in cv.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model = CatBoostClassifier(**params)
        model.fit(X_train, y_train, 
                 eval_set=(X_val, y_val),
                 early_stopping_rounds=50,
                 verbose=False)
        
        preds = model.predict(X_val)
        score = accuracy_score(y_val, preds)
        cv_scores.append(score)
    
    return np.mean(cv_scores)

# CatBoost最適化の実行
print("🔍 CatBoostのハイパーパラメータ最適化を開始...")
study_cat = optuna.create_study(direction='maximize',
                                 sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study_cat.optimize(objective_cat, n_trials=50, show_progress_bar=True)

print(f"\n✅ CatBoost最適化完了")
print(f"   Best CV Score: {study_cat.best_value:.4f}")
print(f"   Best Parameters:")
for key, value in study_cat.best_params.items():
    print(f"     {key}: {value}")

### 5.4 最適化結果の可視化

In [ ]:
# 最適化履歴のプロット
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, study, name in zip(axes, [study_lgb, study_xgb, study_cat], 
                            ['LightGBM', 'XGBoost', 'CatBoost']):
    trials_df = study.trials_dataframe()
    ax.plot(trials_df['number'], trials_df['value'], marker='o', alpha=0.6)
    ax.axhline(study.best_value, color='red', linestyle='--', 
               label=f'Best: {study.best_value:.4f}')
    ax.set_xlabel('Trial')
    ax.set_ylabel('CV Accuracy')
    ax.set_title(f'{name} Optimization History')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# パラメータ重要度の比較
print("\n📊 モデル別の最適CV Accuracy:")
print(f"LightGBM: {study_lgb.best_value:.4f}")
print(f"XGBoost:  {study_xgb.best_value:.4f}")
print(f"CatBoost: {study_cat.best_value:.4f}")

## 🎭 6. アンサンブル学習

3つのモデルを組み合わせてアンサンブルを作成します。

### 6.1 10-Fold Stratified CV with Out-of-Fold Predictions

In [ ]:
def train_with_kfold(X, y, X_test, params_lgb, params_xgb, params_cat, n_splits=10):
    """
    10-Fold Stratified CVでモデルを訓練し、Out-of-Fold予測を取得
    """
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    
    # Out-of-fold predictions用の配列
    oof_lgb = np.zeros(len(X))
    oof_xgb = np.zeros(len(X))
    oof_cat = np.zeros(len(X))
    
    # テスト予測用の配列
    test_preds_lgb = np.zeros(len(X_test))
    test_preds_xgb = np.zeros(len(X_test))
    test_preds_cat = np.zeros(len(X_test))
    
    # CVスコアの記録
    cv_scores_lgb = []
    cv_scores_xgb = []
    cv_scores_cat = []
    
    print(f"\n🔄 {n_splits}-Fold Cross-Validation開始...\n")
    
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y), 1):
        print(f"Fold {fold}/{n_splits}")
        
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # LightGBM
        model_lgb = lgb.LGBMClassifier(**params_lgb)
        model_lgb.fit(X_train, y_train,
                     eval_set=[(X_val, y_val)],
                     callbacks=[lgb.early_stopping(50, verbose=False)])
        oof_lgb[val_idx] = model_lgb.predict_proba(X_val)[:, 1]
        test_preds_lgb += model_lgb.predict_proba(X_test)[:, 1] / n_splits
        score_lgb = accuracy_score(y_val, model_lgb.predict(X_val))
        cv_scores_lgb.append(score_lgb)
        
        # XGBoost
        model_xgb = xgb.XGBClassifier(**params_xgb)
        model_xgb.fit(X_train, y_train,
                     eval_set=[(X_val, y_val)],
                     early_stopping_rounds=50,
                     verbose=False)
        oof_xgb[val_idx] = model_xgb.predict_proba(X_val)[:, 1]
        test_preds_xgb += model_xgb.predict_proba(X_test)[:, 1] / n_splits
        score_xgb = accuracy_score(y_val, model_xgb.predict(X_val))
        cv_scores_xgb.append(score_xgb)
        
        # CatBoost
        model_cat = CatBoostClassifier(**params_cat)
        model_cat.fit(X_train, y_train,
                     eval_set=(X_val, y_val),
                     early_stopping_rounds=50,
                     verbose=False)
        oof_cat[val_idx] = model_cat.predict_proba(X_val)[:, 1]
        test_preds_cat += model_cat.predict_proba(X_test)[:, 1] / n_splits
        score_cat = accuracy_score(y_val, model_cat.predict(X_val))
        cv_scores_cat.append(score_cat)
        
        print(f"  LightGBM: {score_lgb:.4f} | XGBoost: {score_xgb:.4f} | CatBoost: {score_cat:.4f}")
    
    print(f"\n✅ CV完了")
    print(f"\n📊 平均CVスコア:")
    print(f"  LightGBM: {np.mean(cv_scores_lgb):.4f} (+/- {np.std(cv_scores_lgb):.4f})")
    print(f"  XGBoost:  {np.mean(cv_scores_xgb):.4f} (+/- {np.std(cv_scores_xgb):.4f})")
    print(f"  CatBoost: {np.mean(cv_scores_cat):.4f} (+/- {np.std(cv_scores_cat):.4f})")
    
    return {
        'oof': {'lgb': oof_lgb, 'xgb': oof_xgb, 'cat': oof_cat},
        'test': {'lgb': test_preds_lgb, 'xgb': test_preds_xgb, 'cat': test_preds_cat},
        'scores': {'lgb': cv_scores_lgb, 'xgb': cv_scores_xgb, 'cat': cv_scores_cat}
    }

# 10-Fold CVの実行
results = train_with_kfold(
    X, y, X_test,
    params_lgb={**study_lgb.best_params, 'objective': 'binary', 'metric': 'binary_logloss', 
                'verbosity': -1, 'random_state': RANDOM_STATE},
    params_xgb={**study_xgb.best_params, 'objective': 'binary:logistic', 'eval_metric': 'logloss',
                'use_label_encoder': False, 'random_state': RANDOM_STATE},
    params_cat={**study_cat.best_params, 'loss_function': 'Logloss', 'eval_metric': 'Accuracy',
                'random_state': RANDOM_STATE, 'verbose': False},
    n_splits=10
)

### 6.2 重み付きアンサンブル

In [ ]:
def find_best_weights(oof_lgb, oof_xgb, oof_cat, y_true):
    """
    最適な重みを探索
    """
    from scipy.optimize import minimize
    
    def objective(weights):
        # 重み付き平均
        ensemble = weights[0] * oof_lgb + weights[1] * oof_xgb + weights[2] * oof_cat
        preds = (ensemble > 0.5).astype(int)
        return -accuracy_score(y_true, preds)  # 最小化するため負にする
    
    # 制約: 重みの合計が1
    constraints = {'type': 'eq', 'fun': lambda w: w.sum() - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    # 初期値: 均等な重み
    initial_weights = np.array([1/3, 1/3, 1/3])
    
    result = minimize(objective, initial_weights, method='SLSQP', 
                     bounds=bounds, constraints=constraints)
    
    return result.x

# 最適な重みを探索
best_weights = find_best_weights(
    results['oof']['lgb'],
    results['oof']['xgb'],
    results['oof']['cat'],
    y
)

print(f"\n🎯 最適な重み:")
print(f"  LightGBM: {best_weights[0]:.3f}")
print(f"  XGBoost:  {best_weights[1]:.3f}")
print(f"  CatBoost: {best_weights[2]:.3f}")

# アンサンブル予測
oof_ensemble = (best_weights[0] * results['oof']['lgb'] + 
                best_weights[1] * results['oof']['xgb'] + 
                best_weights[2] * results['oof']['cat'])

test_ensemble = (best_weights[0] * results['test']['lgb'] + 
                 best_weights[1] * results['test']['xgb'] + 
                 best_weights[2] * results['test']['cat'])

# アンサンブルのCVスコア
ensemble_preds = (oof_ensemble > 0.5).astype(int)
ensemble_score = accuracy_score(y, ensemble_preds)

print(f"\n📊 アンサンブルCVスコア: {ensemble_score:.4f}")

# 個別モデルとアンサンブルの比較
oof_lgb_preds = (results['oof']['lgb'] > 0.5).astype(int)
oof_xgb_preds = (results['oof']['xgb'] > 0.5).astype(int)
oof_cat_preds = (results['oof']['cat'] > 0.5).astype(int)

print(f"\n📈 個別モデルとの比較:")
print(f"  LightGBM単体:  {accuracy_score(y, oof_lgb_preds):.4f}")
print(f"  XGBoost単体:   {accuracy_score(y, oof_xgb_preds):.4f}")
print(f"  CatBoost単体:  {accuracy_score(y, oof_cat_preds):.4f}")
print(f"  アンサンブル:   {ensemble_score:.4f} ⭐")

## 📤 7. Kaggle提出ファイルの作成

最終的な予測をKaggle提出形式で保存します。

In [ ]:
# 提出用のディレクトリ作成
submission_dir = Path('../submissions')
submission_dir.mkdir(exist_ok=True)

# 予測を0/1に変換
final_predictions = (test_ensemble > 0.5).astype(int)

# 提出ファイルの作成
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': final_predictions
})

# 保存
submission_path = submission_dir / 'titanic_ensemble_submission.csv'
submission.to_csv(submission_path, index=False)

print(f"✅ 提出ファイル作成完了: {submission_path}")
print(f"\n📊 予測分布:")
print(submission['Survived'].value_counts())
print(f"\n生存予測率: {submission['Survived'].mean():.2%}")

# 最初の10行を表示
print(f"\n最初の10行:")
display(submission.head(10))

## 📊 8. 結果の可視化と分析

In [ ]:
# CV Scores の箱ひげ図
fig, ax = plt.subplots(figsize=(10, 6))

data_to_plot = [
    results['scores']['lgb'],
    results['scores']['xgb'],
    results['scores']['cat']
]

bp = ax.boxplot(data_to_plot, labels=['LightGBM', 'XGBoost', 'CatBoost'],
                patch_artist=True)

for patch, color in zip(bp['boxes'], ['skyblue', 'lightgreen', 'lightcoral']):
    patch.set_facecolor(color)

ax.set_ylabel('Accuracy')
ax.set_title('10-Fold CV Scores by Model')
ax.grid(True, alpha=0.3)
plt.show()

# Confusion Matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y, ensemble_preds)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Ensemble Model - Confusion Matrix')
plt.show()

print(f"\n📊 Classification Report:")
print(classification_report(y, ensemble_preds, target_names=['Not Survived', 'Survived']))

## 🎓 9. まとめと次のステップ

### 📈 達成したこと

1. ✅ 高度な特徴量エンジニアリング（Title、Deck、Ticket prefix、Family sizeなど）
2. ✅ Optunaによるハイパーパラメータ最適化
3. ✅ 10-Fold Stratified CVによる堅牢な検証
4. ✅ 3モデルの重み付きアンサンブル
5. ✅ Kaggle提出ファイルの作成

### 🎯 期待されるスコア

- **Local CV Accuracy**: 0.85前後
- **Kaggle Public LB**: 0.79+（Top 30%圏内）

### 🚀 さらにスコアを上げるには

1. **追加の特徴量エンジニアリング**:
   - より複雑な交互作用特徴量
   - 名前の長さ、レア度など
   - ドメイン知識に基づく特徴量

2. **アンサンブルの改善**:
   - Stackingの導入（次のノートブック22で学習）
   - より多様なモデルの追加（Neural Network、SVMなど）

3. **Pseudo-labeling**:
   - テストデータの予測を訓練データに追加

4. **Calibration**:
   - Platt Scalingによる確率の調整

### 📚 次のノートブック

- **ノートブック18**: House Prices回帰問題でGBDT回帰を学ぶ
- **ノートブック19**: Store Demand時系列予測に挑戦
- **ノートブック20**: Optunaをさらに深く学ぶ

---

**🎉 お疲れ様でした！Kaggleで実際に提出してスコアを確認してみましょう！**

## 📝 付録: よくある質問（FAQ）

### Q1: CVスコアとLBスコアの差が大きい場合は？

**A**: 過学習の可能性があります。以下を試してください：
- 正則化パラメータ（`reg_alpha`, `reg_lambda`）を増やす
- `n_estimators`を減らす
- より多くのFold数でCVを行う

### Q2: Optunaの最適化に時間がかかりすぎる

**A**: 以下の方法で高速化できます：
- `n_trials`を減らす（50 → 20など）
- CVのFold数を減らす（10 → 5など）
- Pruningを有効にする（Optuna MedianPruner）

### Q3: アンサンブルで精度が下がった

**A**: モデルが類似している可能性があります：
- より多様なモデルを追加（Neural Network、SVMなど）
- 異なる特徴量セットで訓練したモデルを使う
- Stackingを試す

### Q4: メモリ不足エラーが出る

**A**: 以下を試してください：
- 不要な変数を削除（`del`や`gc.collect()`）
- バッチ処理を使う
- データ型を最適化（`float64` → `float32`など）

---

## 🔗 参考リンク

- [Kaggle Titanic Competition](https://www.kaggle.com/c/titanic)
- [LightGBM Documentation](https://lightgbm.readthedocs.io/)
- [XGBoost Documentation](https://xgboost.readthedocs.io/)
- [CatBoost Documentation](https://catboost.ai/docs/)
- [Optuna Documentation](https://optuna.org/)

---

**Happy Kaggling! 🚀**